# Rain Garden Map: Flushing, Queens

## Setup

### Imports

In [5]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import re

### Data Read-In

In [8]:
# Read in CSV file and remove rows 0 through 2
df = pd.read_csv("GI Assets-Grid view.csv")
df = df.iloc[3:]
df.head()

,ID,Description,Asset type,Status,Photo,Program Area,Coordinates,Record link,Stewardship,Associated partner,Garden Stewardship copy
3,753B,NaN,Rain Garden,Regular stewardship,753b.jpeg (https://v5.airtableusercontent.com/...,Corona,"40°44'45""N 73°52'05""W",NaN,"753B on Fri 12/05,753B on Fri 10/24",CASA,NaN
4,760A,NaN,Rain Garden,Regular stewardship,760a.jpeg (https://v5.airtableusercontent.com/...,Corona,"40°44'46""N 73°51'59""W",NaN,"760A on Fri 11/14,760A on Fri 12/05,760A on Fr...",CASA,NaN
5,760B,NaN,Rain Garden,Regular stewardship,NaN,Corona,"40°44'46""N 73°51'59""W",NaN,760B on Fri 12/05,CASA,NaN
6,764A,NaN,Rain Garden,Regular stewardship,764a.jpeg (https://v5.airtableusercontent.com/...,Corona,"40°44'49""N 73°52'05""W",NaN,"764A on Fri 12/05,764A on Fri 10/24",CASA,NaN
7,766B,NaN,Rain Garden,Regular stewardship,766b.jpeg (https://v5.airtableusercontent.com/...,Corona,"40°44'48""N 73°52'06""W",NaN,"766B on Fri 12/05,766B on Fri 10/24",CASA,NaN


In [9]:
def dms_to_dd(dms):
    # Match DMS like 40°44'45"N or 73°52'05"W
    match = re.match(r"""(\d+)°(\d+)'([\d.]+)"([NSEW])""", dms)
    deg, minute, sec, direction = match.groups()
    dd = float(deg) + float(minute)/60 + float(sec)/3600
    if direction in ["S", "W"]:
        dd *= -1
    return dd

# Convert DMS to decimal degrees
# Split coordinate column
coords = df["Coordinates"].str.split(" ", expand=True)
df["lat"] = coords[0].apply(dms_to_dd)
df["lon"] = coords[1].apply(dms_to_dd)

# Drop empty columns
df = df.replace(r"^\s*$", pd.NA, regex=True)
df = df.dropna(axis=1, how="all")

# Create geometry
gdf = gpd.GeoDataFrame(
    df,
    geometry=[Point(xy) for xy in zip(df.lon, df.lat)],
    crs="EPSG:4326"
)

# Export
gdf.to_file("rain_gardens.geojson", driver="GeoJSON")